# Revised case normalization for Hirslanden Aarau 2018

Notes:

- 17 of 17 revised DtoD cases were joined with the BFS cases from the DB.



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden Aarau', '2018', 'Aarau 2018')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Aarau', year='2018', sheet='Aarau 2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-10 13:51:02.240 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 17 cases for Hirslanden Aarau 2018
2022-11-10 13:51:02.245 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:60 - TYPES:
tranche                   string
datum                     string
case_id                   object
patient_id                object
kkik                      string
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string
cw-

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041623461,0001059029,W,77,2,41623461,I7024,I7024,M100,3,F59E,[J4481],[J4483],[],[]
1,0041665678,0002043896,M,82,17,41665678,I5001,I5001,M100,3,F62C,[R18],[],[],[]
2,0041698586,0022177655,M,93,2,41698586,I7024,I7024,M100,3,F59E,[I743],[],[],[]
3,0041719219,0001339311,M,91,4,41719219,I495,I495,M100,3,F71B,[N182],[],[],[]
4,41777951,0022177655,M,93,2,41777951,I7024,I7024,M100,0,F59E,"[N184, T828]",[],[],[]
5,0041648887,0001709901,M,64,8,41648887,I5014,I5014,M100,3,F62C,[E1191],[E1190],[],[]
6,0041709615,0001749027,W,58,5,41709615,N390,N390,M100,3,L63D,[B370],[],[],[]
7,0041772407,0022190089,W,79,9,41772407,N390,N390,M100,3,L63D,[E440],[],[890A32],[]
8,0041769953,0022179314,M,70,6,41769953,N1781,N1781,M100,3,L60D,[J9610],[],[],[]
9,0041803251,0001720953,M,51,8,41803251,I208,I208,M200,3,F06D,[J9580],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.dtypes

2022-11-10 13:51:05.417 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 10112 rows from the DB, for the hospital 'Hirslanden Aarau' in 2018


aimedic_id                 int64
case_id                   object
primary_diagnosis         object
secondary_diagnoses       object
primary_procedure         object
secondary_procedures      object
gender                    object
age_years                  int64
age_days                   int64
gestation_age              int64
duration_of_stay           int64
ventilation_hours          int64
grouper_admission_type    object
admission_date            object
admission_weight           int64
grouper_discharge_type    object
discharge_date            object
dtype: object

In [5]:
if unmatched.shape[0] > 0:
    unmatched

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-10 13:51:06.280 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 17 cases ...
2022-11-10 13:51:07.308 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 17 cases into: 17 revisions, 168 diagnoses rows, 108 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,431229,F59B,2.55,0.97,4,2022-12-31
1,431363,F71A,1.12,1.12,4,2022-12-31
2,431860,F62B,1.30,1.30,4,2022-12-31
3,432553,F03C,4.90,4.90,4,2022-12-31
4,432647,F62B,1.30,1.30,4,2022-12-31
5,433607,F59B,2.55,1.50,4,2022-12-31
6,433746,L63B,1.09,1.09,4,2022-12-31
7,433916,L60C,1.40,1.40,4,2022-12-31
8,434365,I10A,2.66,2.66,4,2022-12-31
9,435248,F06C,3.84,3.84,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-10 13:51:07.327 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 17 cases into the 'Revisions' table ...
2022-11-10 13:51:08.984 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 17 rows from the "Revisions" table, which is about to be updated
2022-11-10 13:51:09.060 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 17 cases into the 'Revisions' table
2022-11-10 13:51:09.061 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 168 rows into the 'Diagnoses' table ...
2022-11-10 13:51:10.569 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 168 rows into the 'Diagnoses' table
2022-11-10 13:51:10.572 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 108 rows into the 'Procedures' ta